In [ ]:
# | default_exp cli

# Command Line Interface

In [ ]:
# | hide
import tempfile
import jupyter_black

from fastcore.test import *
from sal.utils import files

jupyter_black.load()

In [ ]:
# | export
import click

from pathlib import Path
from typing import Any

from sal.core import Data
from sal.utils import is_notebook
from sal.loaders import xml_file_to_data
from sal.templates import MissingTemplateException
from sal.codegen import Sal, Renderer
from sal.templates import TemplateLoader, TemplateRenderer

## Generating code


Up until now, we've been developing the code we need to generate code. Now it's time to wrap that code in a easy to use function to use as a command line interface. This cli will mirror the args of this function so:

- it accepts an xml file path as input
- it accepts a template directory as an input

Also, when a certain template does not exist, it will create one with a default template `Renderer.DEFAULT_TEMPLATE`.

In [ ]:
# | export
def _render(file: str, directory: str, exit=False) -> str | Any:
    try:
        repository = TemplateLoader.from_directory(directory)
        renderer = Renderer(repository=repository, renderer=TemplateRenderer())
        sal = Sal(renderer)
        struct: Data = xml_file_to_data(file)
        return sal.process(struct)
    except MissingTemplateException as e:

        if exit:
            raise RuntimeError(
                f"Exiting after not finding the template twice: {e.name}"
            )

        path = Path(directory) / f"{e.name}.jinja2"
        path.write_text(Renderer.DEFAULT_TEMPLATE)
        return _render(file, directory, exit=True)

In [ ]:
# | hide
destination = "/tmp/hello/a/4output.txt"

model = """
---
reference:  "sigla-{{ node.attrs.name | lower }}-model"
---
class {{ name }}Model(models.Model): # {{ reference }}
    {% for child in children -%}
    {{ child | render }}
    {% endfor %}
"""

field = """
---
reference:  "sigla-{{ node.name | lower }}-model"
---
{{ name }} = models.{{ type | title }}Field() 
"""

base_folder = "/tmp/templates"


xml = f"""
<wrapper>
    <to-file to="{destination}">
        <model name="User">
            <field name="id" type="integer"/>
            <field name="username" type="char"/>
            <field name="email" type="email"/>
        </model>
    </to-file>
</wrapper>
"""

with files(
    {
        f"{base_folder}/model.jinja2": model,
        f"{base_folder}/field.jinja2": field,
        "/tmp/sal.xml": xml,
        destination: "",
    }
):
    _render("/tmp/sal.xml", base_folder)
    result = Path(destination).read_text().strip()

test_eq(
    result,
    """
class UserModel(models.Model): # sigla-user-model
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
""".strip(),
)

---

In [ ]:
# | export
@click.group()
def main() -> None:
    pass

In [ ]:
# | export

# TODO : init command
# - create : sal.xml file
# - create : sal folder
# - create : sal/templates folder


@main.command()
@click.option("--filename", type=click.Path(exists=True), default="./sal.xml")
@click.option("--folder", type=click.Path(exists=True), default="./sal")
def render(filename: str, folder: str) -> None:
    click.echo(f"⚠️ {filename=}")
    click.echo(f"⚠️ {folder=}")
    _render(filename, str(Path(folder) / "templates"))

In [ ]:
# | export
if __name__ == "__main__" and not is_notebook():
    main()

---

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()